# Partial Experiment Loader

This notebooks demos how to create a dataloader for a subset of the Experiment data. This subset is defined either by a list of `frame id intervals`.

In [7]:
import json
from tqdm import tqdm
import numpy as np

from experanto.intervals import (
    TimeInterval,
    find_complement_of_interval_array,
    uniquefy_interval_array,
)

In [8]:
# Data paths

base = '/data/test_upsampling_without_hamming_30.0Hz/'

paths = [
            # 'dynamic26872-17-20-Video-021a75e56847d574b9acbcc06c675055_30hz/screen/combined_meta.json',
            # 'dynamic27204-5-13-Video-021a75e56847d574b9acbcc06c675055_30hz/screen/combined_meta.json',
            'dynamic29515-10-12-Video-021a75e56847d574b9acbcc06c675055_30hz/screen/combined_meta.json',
            # 'dynamic29623-4-9-Video-021a75e56847d574b9acbcc06c675055_30hz/screen/combined_meta.json',
            # 'dynamic29647-19-8-Video-021a75e56847d574b9acbcc06c675055_30hz/screen/combined_meta.json',
            # 'dynamic29712-5-9-Video-021a75e56847d574b9acbcc06c675055_30hz/screen/combined_meta.json',
            # 'dynamic29755-2-8-Video-021a75e56847d574b9acbcc06c675055_30hz/screen/combined_meta.json',
            ]

In [9]:
meta_path = base + paths[0]
with open(meta_path, 'rb') as f:
    meta = json.load(f)
meta['00004']

{'condition_hash': '9NUudHK3AYroIBWxG5ny',
 'first_frame_idx': 1224,
 'image_size': [36, 64],
 'modality': 'video',
 'num_frames': 300,
 'stim_type': 'stimulus.Clip',
 'tier': 'final_test_main',
 'trial_idx': 4}

In [10]:
valid_keys = ['00002','00004','00007','00008','00009','00011','00012','00013','00014','00015','00017','00019','00020','00022','00023','00024','00025','00027','00028','00029','00032','00033','00034','00036','00037',]

In [18]:
def id2interval(id_list, dataset, complement=False):
    '''Convert a list of IDs to intervals.
    Args:
        id_list (list): The list of IDs.
        complement (bool): If True, return the complement of the intervals.
    Returns:
        list: A list of intervals.'''
    
    if not id_list:
        return []
    
    id_list = sorted(id_list)

    if complement:
        all_ids = set(meta.keys())
        used_ids = set(id_list)
        complement_ids = sorted(all_ids - used_ids)
        return id2interval(complement_ids, )
    
    # Should pass dataset
    timestamps = np.load(f"/data/test_upsampling_without_hamming_30.0Hz/{dataset}/screen/timestamps.npy")
    meta_path = f"/data/test_upsampling_without_hamming_30.0Hz/{dataset}/screen/combined_meta.json"
    with open(meta_path, 'rb') as f:
        meta = json.load(f)
    
    intervals = []
    
    for i in range(1, len(id_list)):
        start = meta[id_list[i]]['first_frame_idx']
        end = meta[id_list[i]]['first_frame_idx'] + meta[id_list[i]]['num_frames']
        intervals.append(TimeInterval(timestamps[start], timestamps[end]))  # The start is inclusive and the end is exclusive
    
    valid_intevals = uniquefy_interval_array(intervals)
    
    return valid_intevals

In [ ]:
id_list = valid_keys
dataset = "dynamic29515-10-12-Video-021a75e56847d574b9acbcc06c675055_30hz"
id2interval(id_list=id_list, dataset=dataset)

[TimeInterval(start=1682534622.5913115, end=1682534632.6021829),
 TimeInterval(start=1682534652.623929, end=1682534683.1570945),
 TimeInterval(start=1682534693.167968, end=1682534742.721799),
 TimeInterval(start=1682534752.732674, end=1682534762.7435498),
 TimeInterval(start=1682534772.7544258, end=1682534792.7761786),
 TimeInterval(start=1682534802.787054, end=1682534839.326755),
 TimeInterval(start=1682534849.3376324, end=1682534878.3691757),
 TimeInterval(start=1682534898.3909318, end=1682534928.92411),
 TimeInterval(start=1682534938.9349887, end=1682534959.957831)]

### ToDo: Use the time intervals to dataloader function to complete the demo